In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from Utils.tensor_board import Tensorboard
import os
import torch
import numpy
import random
import argparse
import datetime
from train import Trainer
from Utils.logger import *

from torch.utils.data import DataLoader
from Configs.config import config
from Dataloader.dataset import LAHeartDataset
from Dataloader.dataloader import TwoStreamBatchSampler


def main(args):
    # update the default config with the args
    config.update(vars(args))

    def worker_init_fn(worker_id):
        random.seed(config.seed + worker_id)

    current_time = datetime.datetime.now()


    train_set = LAHeartDataset(
        os.path.join(config.code_path, "Datasets/Left_Atrium/labels"),
        config.data_path,
        split="train",
        config=config,
    )

    # merge both labelled & unlabelled sampler to same batch
    batch_sampler = TwoStreamBatchSampler(
        list(range(config.labeled_num)),
        list(range(config.labeled_num, 80)),
        config.batch_size,
        int(config.batch_size / 2),
    )

    train_loader = DataLoader(
        train_set,
        batch_sampler=batch_sampler,
        num_workers=config.num_workers,
        pin_memory=False,
        worker_init_fn=worker_init_fn,
    )

    val_dataset = LAHeartDataset(
        os.path.join(config.code_path, "Datasets/Left_Atrium/labels"),
        config.data_path,
        split="eval",
        num=None,
        config=config,
    )
    return val_dataset
    config.iter_per_epoch = len(train_loader)
    config.n_epochs = config.max_iterations // len(train_loader) + 1
    config.unlabeled_num = len(train_set) - config.labeled_num

    logger = logging.getLogger("VNET_LA")
    logger.propagate = False
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        filename=f"scores_output/app_{current_time}.log",
    )
    file_handler = logging.FileHandler(f"scores_output/app_{current_time}.log")
    file_handler.setLevel(logging.INFO)  
    file_handler.setFormatter(
        logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
    )
    logger.addHandler(file_handler)

    logger.info(
        "training with {} epochs [{} iters]".format(
            config.n_epochs, config.iter_per_epoch * config.n_epochs
        )
    )
    logger.warning(
        "running time: " + datetime.datetime.now().strftime(" [%H:%M] %d/%m/%y")
    )
    logger.warning(
        "supervised sample: {}, unsupervised sample: {}".format(
            config.labeled_num, config.unlabeled_num
        )
    )
    logger.critical(
        "architecture: {}, backbone: {}".format(
            args.architecture, "nothing" if args.backbone is None else args.backbone
        )
    )
    tensorboard = Tensorboard(config=config)
    trainer = Trainer(
        config,
        train_loader=train_loader,
        valid_set=val_dataset,
        logger=logger,
        my_wandb=tensorboard,
    )
    trainer.run()
    return


if __name__ == "__main__":

    class CmdLineVar:
        pass

    cmd_line_var = CmdLineVar()
    cmd_line_var.architecture = "vnet"
    cmd_line_var.backbone = None
    cmd_line_var.unsup_weight = 0.3
    cmd_line_var.labeled_num = 8
    cmd_line_var.max_iterations = 9000

    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False

    random.seed(config.seed)
    numpy.random.seed(config.seed)
    torch.manual_seed(config.seed)
    torch.cuda.manual_seed(config.seed)

    train_set = main(cmd_line_var)


In [9]:
train_set[0][2].shape

torch.Size([64])

In [ ]:
for batch_idx, (normal_batch, cons_batch) in enumerate(train_set):
    print(batch_idx, normal_batch, cons_batch)
    

In [13]:
import torch
import torch.nn as nn

# Define your model
class LinearModel(nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        self.linear1 = nn.Linear(64, 128)
        self.linear2 = nn.Linear(128, 245)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.sigmoid(x)
        return x

# Create an instance of your model
model = LinearModel().to("cuda")

# Define a sample input tensor
sample_input = train_set[0][2].to("cuda")  # Assuming batch size of 1

# Pass the sample input through the model
output = model(sample_input)

# Print the output shape
print("Output shape:", output.shape)
output = torch.reshape(output, (1,1,7,7,5))
output.shape

Output shape: torch.Size([245])


torch.Size([1, 1, 7, 7, 5])

In [2]:
output.shape

torch.Size([1, 1, 7, 7, 5])

In [ ]:
import torch
from Model.Vnet import *
from torchsummary import summary
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

# Instantiate the VNet model
net = VNet()

# Define input sizes for the two inputs
input_size1 = (2, 112, 112, 80)  # Assuming input size for the first input
input_size2 = (64,)  # Assuming input size for the second input

# Move input tensors to the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = net.to(device)
input_tensor1 = torch.randn(*input_size1).to(device)
input_tensor2 = torch.randn(*input_size2).to(device)

# Print the model summary
summary(net, input_size=[input_tensor1.size(), input_tensor2.size()])


In [ ]:
from torchsummary import summary


# Instantiate the VNet model
model = VNet()
input_size1 = (2, 112, 112, 80)  # Assuming input size for the first input
input_size2 = (64,)  # Assuming input size for the second input

# Move input tensors to the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
input_tensor1 = torch.randn(*input_size1).to(device)
input_tensor2 = torch.randn(*input_size2).to(device)
out = model(input_tensor1, input_tensor2)